# Preliminary Operations

In order to get started and connect to the kaggle API, I need a kaggle token for authentication in the path `./kaggle/kaggle.json`. Hence, I upload the token from my local machine and then create a new directory for it on the remote colab machine. 

In [1]:
import os
from google.colab import files

if 'kaggle.json' not in os.listdir():
  files.upload()

  !mkdir ~/.kaggle
  !cp kaggle.json ~/.kaggle/
  !chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


Once this is done, I can proceed to download the zipped dataset and unzip it. 

In [2]:
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

In [3]:
import zipfile

if 'Questions.csv.zip' not in os.listdir():
  api.dataset_download_file("stackoverflow/stacksample", file_name="Questions.csv", path="./")

if 'Questions.csv' not in os.listdir():
  with zipfile.ZipFile('Questions.csv.zip', 'r') as zipref:
      zipref.extractall('./')

Now I can install PySpark in order to perform well-scaling operations on the dataset and make use of the Spark-Hadoop framework. 

In [4]:
%%capture
!pip install pyspark

# PySpark initialization and dataset loading

In order to handle the dataset, I start a sparkContext and a sparkSession over that context. Note that a SparkContext is unique of each runtime, but one could define multiple SparkSession objects on the same SparkContext one. 

In [6]:
import pyspark
from pyspark import SQLContext
from pyspark.sql import SparkSession

sc = pyspark.SparkContext('local[*]')
spark = SparkSession(sc)
dataset_path = './Questions.csv'

Let's load the dataset and create a `pyspark.sql.dataframe.DataFrame` out of it. A pyspark dataframe allows to work on relational tables in a more readable and optimized way (thanks to Catalyst) then using RDDs. \\
As the entries listed in the file may contain escape characters, I need to specify some more options in order to read the dataset properly. 

In [7]:
df  = spark.read.format('csv') \
                .option('header', True) \
                .option('multiline', True) \
                .option('quote', '"') \
                .option('escape', '"') \
                .load(dataset_path)
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- OwnerUserId: string (nullable = true)
 |-- CreationDate: string (nullable = true)
 |-- ClosedDate: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Body: string (nullable = true)



Of the dataset above, only `Id` and `Body` are kept for further analysis. As there are no null observations (as specified on the [kaggle page](https://www.kaggle.com/stackoverflow/stacksample?select=Questions.csv), the dataset can be kept as it is. \\
More, to make computations faster, I selected only a tenth of the dataset to perform the computations, sampling without replacement.

In [8]:
seed = 1444
fraction = 0.1

df1 = df.select(*['Id', 'Body']).sample(fraction, seed)

N = df1.count()
print('Number of sampled records: ', N)

Number of sampled records:  126474


# Extracting code from  text

As this notebook is made to handle and study StackOverflow questions and that many of those questions contain code sections, it is meaningful to run a parallel analysis on the code found in a question. \\
To do so, I define a udf (User Defined Function) using the `udf` decorator of PySpark. \\
Note that not all of the questions contain code. \\

In [9]:
import re
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, col

@udf(StringType())
def find_code(text):
  """
  Returns the portion of text delimited in two <code> tags
  """
  codes = re.findall(re.compile(r'<code>(.+?)<\/code>', re.DOTALL), text)
  return ' '.join(codes) if codes is not None else ' '  

@udf(StringType())
def remove_code(text):
  """
  Returns a text without sections delimited by <code> tags
  """

  # If there is no code in the text, return the text itself
  if not re.findall(re.compile(r'<code>(.+?)<\/code>', re.DOTALL), text): 
    return text
  else:
    not_codes = re.findall(re.compile(r'^(.+?)<code>', re.DOTALL), text) \
              + re.findall(re.compile(r'<\/code>(.+?)<code>', re.DOTALL), text) \
              + re.findall(re.compile(r'<\/code>(?!.+<code>)(.+?)$', re.DOTALL), text)          
    return ' '.join(not_codes) if not_codes is not None else ' '  

def pre_process(df):
  """
  Applies find_code and remove_code to a DataFrame
  """

  temp = df.withColumn('Code', find_code(col('Body'))) \
           .withColumn('Body_temp', remove_code(col('Body'))) 
  return temp.drop('Body').withColumnRenamed('Body_temp', 'Body')

In [10]:
df2 = pre_process(df1)

Ncode = df2.where(df2.Code != '').count()
print(f'There are {Ncode} questions with code, out of {N}. ',
      f'That is, {float(Ncode/N)*100:.01f}% of the whole dataset')

df2.printSchema()

There are 93536 questions with code, out of 126474.  That is, 74.0% of the whole dataset
root
 |-- Id: string (nullable = true)
 |-- Code: string (nullable = true)
 |-- Body: string (nullable = true)



# Tokenization

Now I come to the interesting part of the analysis, that is to transform the textual data in such a way to be comparable. To do so, I first tokenized the `Body` and `Code` columns making use of a naive, yet fast, 1-gram tokenizer. \\
Please note that code and pure textual data are tokenized in a different way. \\
In order to improve the analysis, stopwords are deleted from the text. 

In [11]:
import nltk

nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english') \
            + ['pre']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
from pyspark.sql.types import ArrayType
from pyspark.sql.functions import col

@udf(ArrayType(elementType = StringType()))
def code_tokenizer(text):
  return [s for s in re.split('[^a-zA-Z.;:]', text) if len(s)>1 and not s.isdigit()]

@udf(ArrayType(elementType = StringType()))
def body_tokenizer(text):
  return [s for s in re.split('[^a-zA-Z]', text.lower()) if len(s)>1 and not s.isdigit() and s not in stopwords]

def tokenize(df):
  """
  Applies code_tokenizer and body_tokenizer to a DataFrame
  """
  return df.withColumn('tkBody', body_tokenizer(col('Body'))) \
           .withColumn('tkCode', code_tokenizer(col('Code'))) \
           .select(*['Id', 'tkBody', 'tkCode'])

In [13]:
df3 = tokenize(df2)
df3.printSchema()

root
 |-- Id: string (nullable = true)
 |-- tkBody: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tkCode: array (nullable = true)
 |    |-- element: string (containsNull = true)



# Vectorization

As idf scores are costly to compute on large dataframes (as they require a global read of the dataframe), I used only `tf` in the vectorization. Although this is not best practice, it still delivers consistent results, especially because the stopwords were already deleted from the documents. 

In [14]:
import numpy as np
from pyspark.sql.types import MapType, DoubleType
from pyspark.sql.functions import col
# Counter objects work with hashable types and are much faster than dictionaries
# more, Counter is a subclass of dict, so every method still works
import collections 

def tf(tokens):
  c = dict(collections.Counter(tokens))
  for k, v in c.items():
    c[k] = float(v/len(tokens))
  return c

def drop(d, keys):
  for k in keys:
    d.pop(k)
  return d

def vectorize(df, columns):
  """
  Vectorizes the `columns` fields of each entry of a DataFrame, then drops `columns`
  """
  a = df.rdd.map(lambda row: row.asDict())
  a = a.map(lambda d: d.update({f'vec{column}' : tf(d[column]) for column in columns} ) or d)  
  a = a.map(lambda d: drop(d, columns))
  return a.toDF()

In [15]:
df4 = vectorize(df3, columns=['tkBody', 'tkCode'])
df4.printSchema()

root
 |-- Id: string (nullable = true)
 |-- vectkBody: map (nullable = true)
 |    |-- key: string
 |    |-- value: double (valueContainsNull = true)
 |-- vectkCode: map (nullable = true)
 |    |-- key: string
 |    |-- value: double (valueContainsNull = true)



# Similarity evaluation

After vectorizing the documents, finding similar pairs is a matter of defining a similarity measure and comparing. \\
I first show a way to find the most similar record to a given one. \\

In [16]:
def dot(dict1, dict2):
  return np.array([dict1[w]*dict2[w] for w in set(dict1.keys()).intersection(set(dict2.keys()))]).sum()

def norm(dict1):
  return np.linalg.norm(np.array(tuple(dict1.values())))

def cos_sim(dict1, dict2):
  """
  If both dictionaries are non empty returns the similarity score. Otherwise, returns -1
  """
  return float(dot(dict1, dict2) / (norm(dict1)*norm(dict2))) if len(dict1.keys())*len(dict2.keys()) > 0 else -1.0

def find_record(df, id):
  """
  Returns the record (assumed existing) in`df` that has the specified `id`, as a dictionary
  """
  return df.where(df.Id == str(id)).take(1)[0].asDict()

def hmean(x, y):
  """
  To be used as a standalone function. 
  Handles non-positive values of `x` and `y`. 
  """
  # if any is 0, return 0
  if x*y == 0:
    return 0
  # if one is positive and the other one is negative, return the positive one
  elif x*y < 0:
    return x if x > 0 else y 
  # if both are negative, return -1 (both negative means that a document lacks a 
  # code while the other lacks a body)
  elif x < 0 and y < 0:
    return -1
  # if they are both positive, return the harmonic mean
  else:
    return 2/(1/x + 1/y) 

@udf(DoubleType())
def udf_hmean(x, y):
  """
  To be used in a DataFrame transformation
  Handles non-positive values of `x` and `y`. 
  """
  # if any is 0, return 0
  if x*y == 0:
    return 0
  # if one is positive and the other one is negative, return the positive one
  elif x*y < 0:
    return x if x > 0 else y 
  # if both are negative, return -1 (both negative means that a document lacks a 
  # code while the other lacks a body)
  elif x < 0 and y < 0:
    return -1
  # if they are both positive, return the harmonic mean
  else:
    return 2/(1/x + 1/y) 

def compare_with(record, d):
  """
  Computes the cosine similarity of two dictionaries
  """
  a = cos_sim(record['vectkBody'], d['vectkBody'])
  b = cos_sim(record['vectkCode'], d['vectkCode'])
  out = {
    'Id' : d['Id'], 
    f'BodySim_with_{record["Id"]}' : a,
    f'CodeSim_with_{record["Id"]}' : b, 
    f'Similarity_with_{record["Id"]}' : hmean(a, b)
  }
  return out

def compute_similarities_with(record, df):
  """
  Applies compare_with to the DataFrame
  """
  r_id = record['Id']
  return df.rdd.map(lambda row: row.asDict()) \
            .map(lambda d: compare_with(record, d)) \
            .toDF().select('Id', 
                           f'BodySim_with_{r_id}', 
                           f'CodeSim_with_{r_id}', 
                           f'Similarity_with_{r_id}')

@udf(DoubleType())
def compare(d1, d2):
  """
  Returns the cosine similarity of two dictionaries
  """
  return cos_sim(d1, d2)

def pair_cos_sim(df):
  """
  Returns a DataFrame whose entries are pair of entries of `df`. 
  Defines new columns to store similarity measures. 
  """
  pair_df = df.withColumnRenamed('Id', 'Id1') \
              .withColumnRenamed('vectkBody', 'vectkBody1') \
              .withColumnRenamed('vectkCode', 'vectkCode1') \
              .crossJoin(df.withColumnRenamed('Id', 'Id2') \
                           .withColumnRenamed('vectkBody', 'vectkBody2') \
                           .withColumnRenamed('vectkCode', 'vectkCode2')) 
  pair_df = pair_df.where(pair_df.Id1 < pair_df.Id2)

  return pair_df.withColumn('BodySim', compare(col('vectkBody1'), col('vectkBody2'))) \
                .withColumn('CodeSim', compare(col('vectkCode1'), col('vectkCode2'))) \
                .withColumn('Similarity', udf_hmean(col('BodySim'), col('CodeSim'))) \
                .select('Id1', 'Id2', 'BodySim', 'CodeSim', 'Similarity')

# One vs all comparison

I select a record to compare the others with. 

In [17]:
id = 13160
record = find_record(df4, id)
original = find_record(df, id)

In [18]:
df5 = compute_similarities_with(record, df4)
df5.printSchema()

root
 |-- Id: string (nullable = true)
 |-- BodySim_with_13160: double (nullable = true)
 |-- CodeSim_with_13160: double (nullable = true)
 |-- Similarity_with_13160: double (nullable = true)



I can visualize what is the most similar record to `13160`, sorting on the `Similarity_with_13160` column in descending order. The result is record `10560310`. \\
Finally, I can visualize the titles and bodies of both records to check if their similarity is reasonable. 

In [19]:
column = col(f'Similarity_with_{record["Id"]}').desc()
df6 = df5.sort(column)
most_sim = df6.take(2)[1].asDict()

In [20]:
most_sim

{'BodySim_with_13160': 0.44539933408304455,
 'CodeSim_with_13160': -1.0,
 'Id': '10560310',
 'Similarity_with_13160': 0.44539933408304455}

In [21]:
id1 = record['Id']
id2 = most_sim['Id']
result = df.where(col('Id').isin({id1, id2})) \
           .rdd.map(lambda row: row.asDict()).collect()
for d in result:
  print(d['Id'], '\n', d['Title'])

13160 
 Best practice for webservices
10560310 
 How do I create unicode characters with variable numbers?


In [22]:
a = result[0]
print('---- Id: ', a['Id'])
print('---- Title: ', a['Title']) 
print(a['Body'])

---- Id:  13160
---- Title:  Best practice for webservices
<p>I've created a webservice and when I want to use its methods I instantiate it in the a procedure, call the method, and I finally I dispose it, however I think also it could be okay to instantiate the webservice in the "private void Main_Load(object sender, EventArgs e)" event.</p>

<p>The thing is that if I do it the first way I have to instantiate the webservice every time I need one of its methods but in the other way I have to keep a webservice connected all the time when I use it in a form for example. </p>

<p>I would like to know which of these practices are better or if there's a much better way to do it</p>

<p><strong>Strategy 1</strong></p>

<pre><code>private void btnRead_Click(object sender, EventArgs e)
{
    try
    {
        //Show clock
        this.picResult.Image = new Bitmap(pathWait);

        Application.DoEvents();

        //Connect to webservice
        svc = new ForPocketPC.ServiceForPocketPC();
    

In [23]:
a = result[1]
print('---- Id: ', a['Id'])
print('---- Title: ', a['Title']) 
print(a['Body'])

---- Id:  10560310
---- Title:  How do I create unicode characters with variable numbers?
<p>Basically what I want to do is <strong>print u'\u1001'</strong>, but I do not want the <strong>1001</strong> hardcoded. Is there a way I can use a variable or string for this? Also, it would be nice if I could retrieve this code again, when I use the output as input.</p>



# All vs all comparison

After finding similar items to a given one, let's find the most similar pair of items in the dataset. \\
To do so, I take an even smaller data sample (as there are at least `n*(n-1) / 2` similarity measurements to do, one for each pair of non identical records). 

In [24]:
df8 = df4.sample(fraction=1e-2, seed=seed)

In [25]:
df9 = pair_cos_sim(df8)

colOrder = col('Similarity').desc()
dfA = df9.sort(colOrder)

dfA.show()

+--------+--------+------------------+------------------+------------------+
|     Id1|     Id2|           BodySim|           CodeSim|        Similarity|
+--------+--------+------------------+------------------+------------------+
|27293230| 6547590|0.7826237921249267|              -1.0|0.7826237921249267|
|12022590| 6547590|0.7791278649137494|              -1.0|0.7791278649137494|
|27419030|30286120|0.7778174593052023|              -1.0|0.7778174593052023|
|33603060|39419400| 0.777211966942379|              -1.0| 0.777211966942379|
|28426700| 6547590|0.7679919555800158|              -1.0|0.7679919555800158|
|10770160|21449160|0.7658540875486514|              -1.0|0.7658540875486514|
|30286120|33603060|0.7602631123499285|              -1.0|0.7602631123499285|
|15821980|36147580|0.7448452997421312|              -1.0|0.7448452997421312|
|30286120|39419400|0.7293476893416829|              -1.0|0.7293476893416829|
|26807840|27293230|0.7219073890002342|              -1.0|0.7219073890002342|

## Results

In [26]:
dfA.printSchema()

root
 |-- Id1: string (nullable = true)
 |-- Id2: string (nullable = true)
 |-- BodySim: double (nullable = true)
 |-- CodeSim: double (nullable = true)
 |-- Similarity: double (nullable = true)



The most similar are `27293230` and `6547590`. Let's see them 

In [27]:
id4 = 27293230
id5 = 6547590

result1 = df.where(col('Id').isin({id4, id5})) \
            .rdd.map(lambda row: row.asDict()).collect()

for d in result1: 
  print(d['Id'], d['Title'])

6547590 EditText in Google Android
27293230 How to get index of UI?


In [28]:
a = result1[0]
print('---- Id: ', a['Id'])
print('---- Title: ', a['Title']) 
print(a['Body'])

---- Id:  6547590
---- Title:  EditText in Google Android
<p>I want to create the <strong>EditText</strong> which allow <strong>Numeric Values</strong>, <strong>Comma</strong>  and <strong>Delete</strong> and other values are ignore.</p>

<p>So How I can achieve this by programming code ?</p>

<p>Thanks.</p>



In [29]:
a = result1[1]
print('---- Id: ', a['Id'])
print('---- Title: ', a['Title']) 
print(a['Body'])

---- Id:  27293230
---- Title:  How to get index of UI?
<p>I want to get index of UI. I did like following, but I couldn't. Is there any way?</p>

<p><strong>source:</strong></p>

<pre><code>tell application "System Events"
    tell process "System Preferences"
        index of button 9 of scroll area 1 of window 1
    end tell
end tell
</code></pre>

<p><strong>result:</strong></p>

<pre><code>error
</code></pre>

<p><strong>expected result:</strong></p>

<pre><code>9
</code></pre>



## More interesting results

Let's now compare `32332430` and `39351520`, the pair with non-negatve similarity for both code and body

In [30]:
id6 = 32332430 
id7 = 39351520

result2 = df.where(col('Id').isin({id6, id7})) \
            .rdd.map(lambda row: row.asDict()).collect()

for d in result2: 
  print(d['Id'], d['Title'])

32332430 Fetch array after mysqli_multiple_query() function
39351520 some css not working


In [31]:
a = result2[0]
print('---- Id: ', a['Id'])
print('---- Title: ', a['Title']) 
print(a['Body'])

---- Id:  32332430
---- Title:  Fetch array after mysqli_multiple_query() function
<p>In the following statement I am trying to fetch the result of the subjects table in the widget_crop database. But I am receiving the down error message after using the mysqli_multi_query() function in order to be able to use multiple mysql queries inside my php code . How can i correct this and print the table correctly  ? </p>

<pre><code>&lt;?php
    //1.Create a database connection
    $dbhost = "localhost";
    $dbuser ="widget_cms";
    $dbpass="12605532y";
    $dbname = "widget_crop";
    $connection = mysqli_connect($dbhost,$dbuser,$dbpass,$dbname);

    //test if connection occured
    if(mysqli_connect_errno()){
        die("Database connection failed !".
        mysqli_connect_error().")".mysqli_connect_errno().")"
        );
    }
?&gt;
&lt;?php
    $menu_name = "\"Edit me\"";
    $position=4;
    $visible = 1;
    // Often these are form values in POST
    // 2.Perform a db query
    $quer

In [32]:
a = result2[1]
print('---- Id: ', a['Id'])
print('---- Title: ', a['Title']) 
print(a['Body'])

---- Id:  39351520
---- Title:  some css not working
<p>So I have been following this tutorial for making a calendar widget with html, css, and js but I am stuck on the CSS not working. Did I miss something or miss-type? Where is my html/css going wrong to not end up the same final result? When I try to edit certain CSS it doesn't seem to affect the output.</p>

<p>The tutorial is <a href="http://code.tutsplus.com/tutorials/how-to-build-a-beautiful-calendar-widget--net-12538" rel="nofollow">here</a> for reference to how it is suppose to look like.</p>

<p>Here is my HTML:</p>

<p><div class="snippet" data-lang="js" data-hide="false" data-console="true" data-babel="false">
<div class="snippet-code">
<pre class="snippet-code-css lang-css prettyprint-override"><code>#cal {
  -moz-box-shadow: 0px 3px 3px rgba(0, 0, 0, 0.25);
  -webkit-box-shadow: 0px 3px 3px rgba(0, 0, 0, 0.25);
  margin: 50px auto;
  font: 13px/1.5"Helvetica Neue", Helvatica, Arial, san-serif;
  display: table;
}
#cal .he